In [1]:
import pandas as pd
import numpy as np
import openpyxl

In [2]:
df=pd.read_excel("D:/从c盘转移的桌面/github/Python Data Cleaning/new_concat_大众.xlsx")
df.head()

,Brand,Model family,Model,Vehicle classification,SKU,Type,Internal reference,Trim,Number of electric drive motors,ADAS,...,WLTC range (KM),NEDC range (KM),CLTC range (KM),Fast charging,Smartphone mirroring,Android Auto support,CarPlay support,CarLife support (China only),HiCar support (China only),Smartphone mirroring - Others support
0,大众,NaN,Arteon,中型车,NaN,汽油,Arteon 2022款 2.0T Shooting Brake R-Line,NaN,NaN,●L2,...,0,NaN,0,NaN,●支持CarPlay●支持CarLife●原厂互联/映射,NaN,NaN,NaN,NaN,NaN
1,大众,NaN,Arteon,中型车,NaN,汽油,Arteon 2021款 R Shooting Brake 欧洲版,NaN,NaN,0,...,0,NaN,0,NaN,●支持CarPlay●支持Android Auto,NaN,NaN,NaN,NaN,NaN
2,大众,NaN,ID.4 CROZZ,紧凑型SUV,NaN,纯电动,ID.4 CROZZ 2022款 纯享限量版,NaN,1.0,●L2,...,0,NaN,425,支持,●支持CarPlay●支持CarLife,NaN,NaN,NaN,NaN,NaN
3,大众,NaN,ID.4 CROZZ,紧凑型SUV,NaN,纯电动,ID.4 CROZZ 2022款 标准续航PURE版,NaN,1.0,●L2,...,0,NaN,425,支持,●支持CarPlay●支持CarLife,NaN,NaN,NaN,NaN,NaN
4,大众,NaN,ID.4 CROZZ,紧凑型SUV,NaN,纯电动,ID.4 CROZZ 2022款 长续航PURE+版,NaN,1.0,●L2,...,0,NaN,600,支持,●支持CarPlay●支持CarLife,NaN,NaN,NaN,NaN,NaN


In [3]:
lenth=len(df.columns)
lenth

39

In [4]:
black=df.iloc[0,9][0]
white=df.iloc[60,9][0]


In [5]:
df2=df.copy()

In [6]:
def find_nhappen(text,character,n):
    count=0
    for i,chars in enumerate(text):
        if chars==character:
            count+=1
        if count==n:
            return i 
    return -1

In [7]:
for j in df2.columns:
    # Calculate the maximum count of black stones in column j
    max_black = df[j].fillna('').astype(str).apply(lambda y: y.count(black)).max()
    # Calculate the maximum count of white stones in column j
    max_white = df[j].fillna('').astype(str).apply(lambda y: y.count(white)).max()
    
    # Process black stones
    if max_black != 0:
        for i in range(max_black):
            # Create a new column j+'_black'+str(i+1) in df2 to store processed data
            df2[j+'_black'+str(i+1)] = df2[j].astype(str).fillna('').apply(
                lambda x: '' if x.count(black) < i+1 else
                (
                    # If there is no white stone after the (i+1)-th black stone, keep the rest of the content
                    (x[find_nhappen(x, black, i+1)+1:] if white not in x[find_nhappen(x, black, i+1)+1:] 
                    else x[find_nhappen(x, black, i+1)+1:find_nhappen(x[find_nhappen(x, black, i+1)+1:], white, 1) + find_nhappen(x, black, i+1)+1])
                    if (x.count(black) == max_black) and ((i+1) == max_black) # If the current black stone is the last one in the cell
                    # Find the last black stone and keep the content until the end or the next white stone (x[find_nhappen(x, black, i+1)+1:])
                    else # For general cases, e.g., when it's a black3 column but there are only 2 black stones
                    (
                        x[find_nhappen(x, black, i+1)+1:
                        min(
                            find_nhappen(x[find_nhappen(x, black, i+1)+1:], black, 1), find_nhappen(x[find_nhappen(x, black, i+1)+1:], white, 1)
                        ) + find_nhappen(x, black, i+1) + 1
                    ] if x.count(black) >= i+1 else '') # If the cell contains more than or equal to (i+1) black stones, keep the content; otherwise, set as empty
                ))
    
    # Process white stones
    if max_white != 0:
        for i in range(max_white):
            # Create a new column j+'_white'+str(i+1) in df2 to store processed data
            df2[j+'_white'+str(i+1)] = df2[j].astype(str).fillna('').apply(
                lambda x: '' if x.count(white) < i+1 else
                (
                    # If there is no black stone after the (i+1)-th white stone, keep the rest of the content
                    (x[find_nhappen(x, white, i+1)+1:] if black not in x[find_nhappen(x, white, i+1)+1:]
                    else x[find_nhappen(x, white, i+1)+1:find_nhappen(x[find_nhappen(x, white, i+1)+1:], black, 1) + find_nhappen(x, white, i+1)+1])
                    if (x.count(white) == max_white) and ((i+1) == max_white) # If the current white stone is the last one in the cell
                    # Find the last white stone and keep the content until the end (x[find_nhappen(x, black, i+1)+1:])
                    else # For general cases, e.g., when it's a white3 column but there are only 2 white stones
                    (
                        x[find_nhappen(x, white, i+1)+1:
                        min(
                            find_nhappen(x[find_nhappen(x, white, i+1)+1:], black, 1), find_nhappen(x[find_nhappen(x, white, i+1)+1:], white, 1)
                        ) + find_nhappen(x, white, i+1) + 1
                    ] if x.count(white) >= i+1 else '') # If the cell contains more than or equal to (i+1) white stones, keep the content; otherwise, set as empty
                ))
    
    # Create a new column j+'_other' in df2 to store content that has neither black nor white stones
    df2[j+'_other'] = df2[j].astype(str).fillna('').apply(lambda x: x if (x.count(black) == 0 and x.count(white) == 0) else '')


In [8]:
df3=df2.iloc[:,lenth:]
df3

,Brand_other,Model family_other,Model_other,Vehicle classification_other,SKU_other,Type_other,Internal reference_other,Trim_other,Number of electric drive motors_other,ADAS_black1,...,Smartphone mirroring_black2,Smartphone mirroring_black3,Smartphone mirroring_black4,Smartphone mirroring_white1,Smartphone mirroring_other,Android Auto support_other,CarPlay support_other,CarLife support (China only)_other,HiCar support (China only)_other,Smartphone mirroring - Others support_other
0,大众,nan,Arteon,中型车,nan,汽油,Arteon 2022款 2.0T Shooting Brake R-Line,nan,nan,L2,...,,,,,,nan,nan,nan,nan,nan
1,大众,nan,Arteon,中型车,nan,汽油,Arteon 2021款 R Shooting Brake 欧洲版,nan,nan,,...,,,,,,nan,nan,nan,nan,nan
2,大众,nan,ID.4 CROZZ,紧凑型SUV,nan,纯电动,ID.4 CROZZ 2022款 纯享限量版,nan,1.0,L2,...,,,,,,nan,nan,nan,nan,nan
3,大众,nan,ID.4 CROZZ,紧凑型SUV,nan,纯电动,ID.4 CROZZ 2022款 标准续航PURE版,nan,1.0,L2,...,,,,,,nan,nan,nan,nan,nan
4,大众,nan,ID.4 CROZZ,紧凑型SUV,nan,纯电动,ID.4 CROZZ 2022款 长续航PURE+版,nan,1.0,L2,...,,,,,,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,大众,nan,高尔夫,紧凑型车,nan,汽油,高尔夫 2023款 280TSI DSG R-Line Lite,nan,nan,,...,,,,,,nan,nan,nan,nan,nan
294,大众,nan,高尔夫,紧凑型车,nan,汽油,高尔夫 2023款 280TSI DSG R-Line,nan,nan,L2,...,,,,,,nan,nan,nan,nan,nan
295,大众,nan,高尔夫,紧凑型车,nan,汽油,高尔夫 2023款 380TSI DSG GTI,nan,nan,L2,...,,,,,,nan,nan,nan,nan,nan
296,大众,nan,高尔夫,紧凑型车,nan,汽油,2021款 改款 380TSI DSG GTI,nan,nan,L2,...,,,,,,nan,nan,nan,nan,nan


In [9]:
df3.to_excel("D:/从c盘转移的桌面/github/Python Data Cleaning/new_concat_大众_result.xlsx", index=False)